# Exploring RAG and QA on locally stored data sample of 1000 articles

In [ ]:
import os
from google.colab import drive
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Get dataframe of locally stored  sample

In [115]:
# loading 1000 art sample from file on Google Drive (when using Colab)
import os
from google.colab import drive

if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')

def load_sample_data():
  data = pd.read_excel('/content/drive/MyDrive/Sci_papers/papers_sample.xlsx')

  data['full_article'] = data['title'] + data['abstract'] + data['full-text']
  return data


In [116]:
data = load_sample_data()
article = data['full_article'][996]
data

,paper_id,title,abstract,full-text,full_article
0,9f05493734433385ff6082f0d4368b93a9a6b3df,Journey of cystatins from being mere thiol pro...,Abstract\n\nCystatins are thiol proteinase inh...,Proteases\n\nProteases are enzymes that irreve...,Journey of cystatins from being mere thiol pro...
1,76b6934f3ead3c415d98b18154c3223178bfbc4f,Dear Editor,NaN,\n\nAccording to the World Health Organization...,NaN
2,97c2f4235923a5366f93d98a48f88b68e1941b21,Accurate Representations of the Microphysical ...,Abstract\n\nAerosols and droplets from expirat...,I. INTRODUCTION\n\nThe transmission of respira...,Accurate Representations of the Microphysical ...
3,d327e343b221e242373be5e86d02a03afe4d250b,Journal Pre-proof RECOMENDACIONES DE CONSENSO ...,Abstract\n\nLa gran afectación pulmonar produc...,INTRODUCCION\n\nLa rápida progresión de la pan...,Journal Pre-proof RECOMENDACIONES DE CONSENSO ...
4,088e0fabcaf75dea9bc0e42ed2a85dc0cf677e02,Myofascial Release of the Hamstrings Improves ...,"Abstract\n\nCitation: Itotani, K.; Kawahata, K...","Introduction\n\nIn 2020, COVID-19 had an impac...",Myofascial Release of the Hamstrings Improves ...
...,...,...,...,...,...
995,0140b20fe9bcda5a1933d8923676e2e503e17a5c,Historical analogies and general theoretical s...,Abstract\n\nThis article discusses the plausib...,Introduction\n\nGeneral theoretical schemes an...,Historical analogies and general theoretical s...
996,646651f4c636a6e8064805dfb4931a078af9e523,Evaluation of different types of face masks to...,Abstract\n\nWe expanded a published mathematic...,\n\nwww.nature.com/scientificreports/ As the C...,Evaluation of different types of face masks to...
997,c2187c600a7b07ce51f310da823fe5bafda1495b,Cardiovascular Imaging: Letter Cardiology Myoc...,NaN,"\n\nDear Editor, We would like to share ideas ...",NaN
998,b37ecbbb10f3440ae7549d18395a2cfa048f142a,Article 593491 and Podda GM (2021) Sensitivity...,Abstract\n\nBackground: SARS-CoV-2-infected su...,\n\nBackground: SARS-CoV-2-infected subjects h...,Article 593491 and Podda GM (2021) Sensitivity...


In [76]:
article

'Evaluation of different types of face masks to limit the spread of SARS-CoV-2: a modeling studyAbstract\n\nWe expanded a published mathematical model of SARS-CoV-2 transmission with complex, agestructured transmission and with laboratory-derived source and wearer protection efficacy estimates for a variety of face masks to estimate their impact on COVID-19 incidence and related mortality in the United States. The model was also improved to allow realistic age-structured transmission with a pre-specified R0 of transmission, and to include more compartments and parameters, e.g. for groups such as detected and undetected asymptomatic infectious cases who mask up at different rates. When masks are used at typically-observed population rates of 80% for those ≥ 65 years and 60% for those < 65 years, face masks are associated with 69% (cloth) to 78% (medical procedure mask) reductions in cumulative COVID-19 infections and 82% (cloth) to 87% (medical procedure mask) reductions in related deat

#Huggin face setup

In [4]:

import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [2]:

!pip install --upgrade -q transformers huggingface_hub

In [5]:
from huggingface_hub import login

login(os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Testing gemma

In [77]:
import torch

In [9]:

model_name = "google/gemma-3-1b-it"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
# Let's load the tokenizer first
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [7]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText, AutoModelForCausalLM, Gemma3ForCausalLM

#processor = AutoProcessor.from_pretrained(model_name, use_fast=True)
model = Gemma3ForCausalLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

In [10]:

prompt = "What can you use an LLM for? Answer:"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_new_tokens=512)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

What can you use an LLM for? Answer:

*   **Content Creation:** Write articles, blog posts, social media captions, emails, scripts, poems, and more.
*   **Summarization:** Condense long texts into shorter, more manageable summaries.
*   **Question Answering:** Provide answers to questions based on provided text or general knowledge.
*   **Translation:** Translate text between different languages.
*   **Code Generation:** Generate code in various programming languages.
*   **Brainstorming:** Generate ideas and explore different perspectives.
*   **Chatbot Development:** Create conversational AI agents.
*   **Creative Writing:** Develop characters, plots, and settings.

Do you want me to elaborate on any of these areas, or perhaps suggest a specific task you'd like me to try?


In [12]:
prompt = f"Please summarise this article: {article}  Answer:"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_new_tokens=512)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Please summarise this article: 
Leadership lessons from administrators, faculty, and students during the COVID-19 pandemic

Abstract

Introduction: Due to the largescale scope of the COVID-19 pandemic, strain on the higher education system in the United States has been extraordinary. Yet, with any crisis, there is the opportunity to learn, grow, and develop new knowledge and strategies to benefit educational programs moving forward. The purpose of this study is to describe the leadership lessons learned by academic pharmacy during the COVID-19 pandemic from the perspective of administrators, faculty, and students.

Introduction

The COVID-19 pandemic has been one of the largest health crises in modern history. The pandemic, caused by a novel coronavirus named SARS-CoV-2, was first identified in Wuhan, China in December 2019. 1 The virus is spread through respiratory droplets that land in the nose or mouth of nearby people, which are then inhaled into the lungs. 2 The spread of the viru

#Testing smaller QA model

In [15]:
model_name = "GanjinZero/biobart-v2-base"
model_name = "razent/SciFive-base-Pubmed_PMC"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [16]:
prompt = f"Please summarise this article: {article}  Answer:"
inputs = tokenizer.encode(prompt, return_tensors="pt")
outputs = model.generate(inputs, max_new_tokens=512)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. The authors declare no conflict of interest. focus groups were conducted in the spring of 2020. focus group members focus group members. focus group members. focus group members also commented on the need for structure. [12]... [   and  and and . This manuscript is a report of the ACPE Board of Directors. and   and. and and COP. 13 and.....  The authors declare no conflict of interest. Yes. Please see the online version of this article. Yes. Please see the online version. The


# Simple RAG

## Processign articles

In [24]:
!pip install openpyxl
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:0

In [52]:
import re
import pandas as pd

import os
from google.colab import drive
import chromadb
from chromadb.utils import embedding_functions

import torch



In [28]:
def preprocess_articles(articles_df, chunk_size=200, overlap=25):
    """
    Split articles from a DataFrame into smaller chunks for better retrieval

    Parameters:
    -----------
    articles_df : pandas.DataFrame
        DataFrame with 'paper_id' and 'full_article' columns
    chunk_size : int
        Target size of chunks in tokens (approximate)
    overlap : int
        Number of tokens to overlap between consecutive chunks

    Returns:
    --------
    list of dict
        List of chunk dictionaries with chunk_id, paper_id, text, and source fields
    """
    chunks = []

    # Iterate through DataFrame rows
    for _, row in articles_df.iterrows():
        paper_id = str(row['paper_id'])
        text = row['full_article']

        if not isinstance(text, str) or not text.strip():
            # Skip empty or non-string entries
            continue

        # Simple paragraph splitting
        paragraphs = re.split(r'\n\s*\n', text)

        # Create chunks with overlap
        current_chunk = ""
        chunk_id = 0

        for para in paragraphs:
            # Clean paragraph
            para = para.strip()
            if not para:
                continue

            # If adding this paragraph exceeds chunk size and we have content,
            # save the current chunk and start a new one
            if len(current_chunk) + len(para) > chunk_size and current_chunk:
                chunks.append({
                    "chunk_id": f"{paper_id}-{chunk_id}",
                    "paper_id": paper_id,
                    "text": current_chunk,
                    "source": paper_id  # For citation tracking
                })

                # Start new chunk with overlap from the end of the previous chunk
                words = current_chunk.split()
                if len(words) > overlap:
                    current_chunk = " ".join(words[-overlap:]) + " " + para
                else:
                    current_chunk = para

                chunk_id += 1
            else:
                # Add to current chunk
                if current_chunk:
                    current_chunk += " " + para
                else:
                    current_chunk = para

        # Don't forget the last chunk
        if current_chunk:
            chunks.append({
                "chunk_id": f"{paper_id}-{chunk_id}",
                "paper_id": paper_id,
                "text": current_chunk,
                "source": paper_id
            })

    return chunks

##Building the vector db / semantic search for RAG

In [41]:
def setup_vector_db(collection_name="covid_research", chunks=None, embedding_model_name="all-MiniLM-L6-v2"):
    """
    Set up a vector database by returning an existing collection or creating a new one
    Always uses PersistentClient for disk storage

    Args:
        chunks: List of text chunks with metadata to add (only needed for new collections)
        embedding_model_name: Name of the SentenceTransformer model to use
        collection_name: Name of the collection to create or retrieve
        db_path: Path to store the database

    Returns:
        tuple: (client, collection)
    """

    db_path="/content/drive/MyDrive/Sci_papers/chroma_db"

    # Ensure drive is mounted
    if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')

    # Create path if it doesn't exist
    os.makedirs(db_path, exist_ok=True)

    # Set up embedding function
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=embedding_model_name
    )

    # Set up persistent client
    client = chromadb.PersistentClient(path=db_path)

    # Get or create collection
    try:
        collection = client.get_collection(
            name=collection_name,
            embedding_function=embedding_function
        )
        print(f"Retrieved existing collection '{collection_name}' with {collection.count()} documents")
    except:
        collection = client.create_collection(
            name=collection_name,
            embedding_function=embedding_function
        )
        print(f"Creating new collection '{collection_name}'")

        # Add documents if provided
        if chunks:
            # Prepare data for insertion
            ids = [chunk["chunk_id"] for chunk in chunks]
            documents = [chunk["text"] for chunk in chunks]
            metadatas = [{"paper_id": chunk["paper_id"], "source": chunk["source"]} for chunk in chunks]

            # Add documents to the collection in batches
            batch_size = 100
            for i in range(0, len(ids), batch_size):
                end = min(i + batch_size, len(ids))
                collection.add(
                    ids=ids[i:end],
                    documents=documents[i:end],
            #skipping metadata        metadatas=metadatas[i:end]
                )
                print(f"Added {end} / {len(ids)} chunks to vector database")

    return client, collection

In [96]:
chunks = preprocess_articles(data)



In [ ]:
%%time
collection_name = "covid_research"
embedding_model = "all-MiniLM-L6-v2"
client, collection = setup_vector_db(collection_name, chunks, embedding_model)



In [45]:
#retrieve collection

client, collection = setup_vector_db(collection_name, None, embedding_model)


Retrieved existing collection 'covid_research' with 14727 documents


In [153]:
def retrieve_chunks(query, collection, top_k=5):
    """
    Retrieve the most relevant chunks from the vector database

    Args:
        query: The user query string
        collection: ChromaDB collection
        top_k: Number of chunks to retrieve

    Returns:
        list: List of retrieved text chunks and their metadata
    """

    # Query the collection
    results = collection.query(
        query_texts=[query],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )

    retrieved_chunks = []
    for i in range(len(results["documents"][0])):
        retrieved_chunks.append({
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i],
            "distance": results["distances"][0][i] if "distances" in results else None
        })

    return retrieved_chunks

def format_context(retrieved_chunks, include_metadata=False):
    """
    Format retrieved chunks into a context string

    Args:
        retrieved_chunks: List of retrieved chunks from retrieve_chunks()

    Returns:
        str: Formatted context string
    """
    context = ""
    for i, chunk in enumerate(retrieved_chunks):
        context += f"\n\nChunk {i+1} "
        if include_metadata:
          context += f"(Source: {chunk['metadata']['source']}):\n"
        context += chunk["text"]

    return context

def generate_answer(query, context, model, tokenizer, max_length=512):
    """
    Generate an answer using a seq2seq model

    Args:
        query: The user query
        context: Retrieved context text
        model: HuggingFace model
        tokenizer: HuggingFace tokenizer
        max_length: Maximum output length

    Returns:
        str: Generated answer
    """
    # Create a prompt that combines the query and context
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    print(f"Input tokenised to {len(inputs[0])} tokens")
    print(f"Model {model.name_or_path} will generate now to: {query}")

    # Generate the output
    with torch.no_grad():
        output_ids = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    # Decode the output
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

In [119]:
%%time
query = """
What were the efficacy of vaccines for various age groups?
"""

retrieved_chunks = retrieve_chunks(query, collection, top_k=5)
retrieved_chunks

CPU times: user 143 ms, sys: 5.03 ms, total: 148 ms
Wall time: 370 ms


[{'text': 'of the elderly. Further studies to determine long-term effectiveness of COVID-19 vaccines in high-risk populations and the safety and effectiveness of additional boosters are needed. Main text:'},
 {'text': 'not always take into consideration the daily life barriers of the population. Easy access to vaccination needs to be accomplished by barrier-free access. Demographic changes In response to public health policies, life expectancy has doubled over the last century and the proportion of the elderly in the population continues to rise. Elderly people are more susceptible to infectious diseases and less responsive to vaccination, as is clearly shown by their responsiveness to the influenza vaccine (Franceschi et al. 2017; Demicheli et al. 2018; Levett-Jones 2020) . This growing situation warrants the adaptation of vaccines to the senescing immune system, thereby introducing a certain degree of personalisation at the scale of an age-defined population. For example, the Sanofi 

In [120]:
generation_model = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [156]:
%%time
query = """
What were the efficacy of vaccines for various age groups?
"""

retrieved_chunks = retrieve_chunks(query, collection, top_k=5)

# Format the context
context = format_context(retrieved_chunks)

# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model google/flan-t5-base will generate now to: 
What were the efficacy of vaccines for various age groups?

CPU times: user 17.8 s, sys: 910 ms, total: 18.7 s
Wall time: 20.2 s


'Protection against SARS-CoV-2 infection is lower among older age groups, while protection against hospitalization remains reasonably good for all age groups, among both partially and fully vaccinated'

In [130]:
query

'\nWhat were the efficacy of vaccines for various age groups?\n'

In [157]:
print(context)



Chunk 1 of the elderly. Further studies to determine long-term effectiveness of COVID-19 vaccines in high-risk populations and the safety and effectiveness of additional boosters are needed. Main text:

Chunk 2 not always take into consideration the daily life barriers of the population. Easy access to vaccination needs to be accomplished by barrier-free access. Demographic changes In response to public health policies, life expectancy has doubled over the last century and the proportion of the elderly in the population continues to rise. Elderly people are more susceptible to infectious diseases and less responsive to vaccination, as is clearly shown by their responsiveness to the influenza vaccine (Franceschi et al. 2017; Demicheli et al. 2018; Levett-Jones 2020) . This growing situation warrants the adaptation of vaccines to the senescing immune system, thereby introducing a certain degree of personalisation at the scale of an age-defined population. For example, the Sanofi Pasteu

In [138]:
answer

'Protection against SARS-CoV-2 infection is lower among older age groups, while protection against hospitalization remains reasonably good for all age groups, among both partially and fully vaccinated'

In [158]:
%%time

query = """
What were the efficacy of vaccines for various age groups?
"""

retrieved_chunks = retrieve_chunks(query, collection, top_k=5)

# Format the context
context = format_context(retrieved_chunks)

# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model google/flan-t5-base will generate now to: 
What were the efficacy of vaccines for various age groups?



'Protection against SARS-CoV-2 infection is lower among older age groups, while protection against hospitalization remains reasonably good for all age groups, among both partially and fully vaccinated'

In [160]:
print(context)



Chunk 1 of the elderly. Further studies to determine long-term effectiveness of COVID-19 vaccines in high-risk populations and the safety and effectiveness of additional boosters are needed. Main text:

Chunk 2 not always take into consideration the daily life barriers of the population. Easy access to vaccination needs to be accomplished by barrier-free access. Demographic changes In response to public health policies, life expectancy has doubled over the last century and the proportion of the elderly in the population continues to rise. Elderly people are more susceptible to infectious diseases and less responsive to vaccination, as is clearly shown by their responsiveness to the influenza vaccine (Franceschi et al. 2017; Demicheli et al. 2018; Levett-Jones 2020) . This growing situation warrants the adaptation of vaccines to the senescing immune system, thereby introducing a certain degree of personalisation at the scale of an age-defined population. For example, the Sanofi Pasteu

## Testing FAISS over Chroma

In [143]:
#!pip install faiss-gpu
!pip install faiss-cpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.3 MB/s eta 0:00:00


In [117]:
data = load_sample_data()
chunks = preprocess_articles(data)

In [145]:
import os
import faiss
import numpy as np
import pickle


In [144]:


def setup_faiss_index(chunks=None, embedding_model_name="all-MiniLM-L6-v2",
                      index_name="covid_research",
                      db_path="/content/drive/MyDrive/Sci_papers/faiss_indexes"):
    """
    Set up a FAISS index by loading an existing one or creating a new one

    Args:
        chunks: List of text chunks with metadata to add (only needed for new indexes)
        embedding_model_name: Name of the SentenceTransformer model to use
        index_name: Name of the index to create or retrieve
        db_path: Path to store the index and metadata

    Returns:
        tuple: (index, documents, metadata)
    """
    # Ensure drive is mounted
    if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')

    # Create path if it doesn't exist
    os.makedirs(db_path, exist_ok=True)

    # Paths for storing the index and metadata
    index_path = os.path.join(db_path, f"{index_name}_index.faiss")
    metadata_path = os.path.join(db_path, f"{index_name}_metadata.pkl")

    # Check if the index exists
    if os.path.exists(index_path) and os.path.exists(metadata_path):
        # Load existing index and metadata
        index = faiss.read_index(index_path)
        with open(metadata_path, 'rb') as f:
            saved_data = pickle.load(f)
            documents = saved_data['documents']
            metadatas = saved_data['metadatas']

        print(f"Loaded existing FAISS index '{index_name}' with {index.ntotal} vectors")
        return index, documents, metadatas

    # Create new index if not found or if chunks are provided
    if chunks is None:
        print(f"No existing index found and no chunks provided. Creating empty index.")
        chunks = []

    # Initialize the embedding model
    model = SentenceTransformer(embedding_model_name)

    # Create new FAISS index (L2 distance)
    dimension = model.get_sentence_embedding_dimension()
    index = faiss.IndexFlatL2(dimension)

    # Prepare data if chunks are provided
    documents = []
    metadatas = []
    embeddings = []

    if chunks:
        # Extract text and metadata
        documents = [chunk["text"] for chunk in chunks]
        metadatas = [{"paper_id": chunk["paper_id"], "source": chunk["source"]} for chunk in chunks]

        # Create embeddings in batches
        batch_size = 32
        print(f"Creating embeddings for {len(documents)} documents...")

        for i in range(0, len(documents), batch_size):
            end = min(i + batch_size, len(documents))
            batch_texts = documents[i:end]
            batch_embeddings = model.encode(batch_texts, convert_to_numpy=True)
            embeddings.append(batch_embeddings)
            print(f"Embedded {end}/{len(documents)} documents")

        # Combine all embeddings and add to index
        if embeddings:
            all_embeddings = np.vstack(embeddings).astype('float32')
            index.add(all_embeddings)

        # Save the index and metadata
        faiss.write_index(index, index_path)
        with open(metadata_path, 'wb') as f:
            pickle.dump({'documents': documents, 'metadatas': metadatas}, f)

        print(f"Created and saved new FAISS index '{index_name}' with {index.ntotal} vectors")

    return index, documents, metadatas



def retrieve_chunks_faiss(query, index, documents, metadatas, embedding_model_name="all-MiniLM-L6-v2", top_k=5):
    """
    Retrieve the most relevant chunks using FAISS

    Args:
        query: The user query string
        index: FAISS index
        documents: List of document texts
        metadatas: List of document metadata
        embedding_model_name: Name of the SentenceTransformer model
        top_k: Number of chunks to retrieve

    Returns:
        list: List of retrieved text chunks and their metadata
    """
    # Initialize the embedding model
    model = SentenceTransformer(embedding_model_name)

    # Create the query embedding
    query_embedding = model.encode([query], convert_to_numpy=True).astype('float32')

    # Search the index
    distances, indices = index.search(query_embedding, k=min(top_k, index.ntotal))

    # Format results
    retrieved_chunks = []
    for i, idx in enumerate(indices[0]):
        if idx < len(documents):  # Safeguard against index out of bounds
            retrieved_chunks.append({
                "text": documents[idx],
                "metadata": metadatas[idx],
                "distance": float(distances[0][i])
            })

    return retrieved_chunks






In [146]:
%%time
embedding_model = "all-MiniLM-L6-v2"
#collection_name = "covid_research"
#client, collection = setup_vector_db(collection_name, chunks, embedding_model)
collection_name = "covid_research_faiss"
index, documents, metadatas = setup_faiss_index(chunks, embedding_model, collection_name)


Creating embeddings for 14727 documents...
Embedded 32/14727 documents
Embedded 64/14727 documents
Embedded 96/14727 documents
Embedded 128/14727 documents
Embedded 160/14727 documents
Embedded 192/14727 documents
Embedded 224/14727 documents
Embedded 256/14727 documents
Embedded 288/14727 documents
Embedded 320/14727 documents
Embedded 352/14727 documents
Embedded 384/14727 documents
Embedded 416/14727 documents
Embedded 448/14727 documents
Embedded 480/14727 documents
Embedded 512/14727 documents
Embedded 544/14727 documents
Embedded 576/14727 documents
Embedded 608/14727 documents
Embedded 640/14727 documents
Embedded 672/14727 documents
Embedded 704/14727 documents
Embedded 736/14727 documents
Embedded 768/14727 documents
Embedded 800/14727 documents
Embedded 832/14727 documents
Embedded 864/14727 documents
Embedded 896/14727 documents
Embedded 928/14727 documents
Embedded 960/14727 documents
Embedded 992/14727 documents
Embedded 1024/14727 documents
Embedded 1056/14727 documents
E

In [147]:
generation_model = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [159]:
%%time
query = """
What were the efficacy of vaccines for various age groups?
"""

retrieved_chunks = retrieve_chunks_faiss(query, index, documents, metadatas, top_k=5)

# Format the context
context = format_context(retrieved_chunks)

# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model google/flan-t5-base will generate now to: 
What were the efficacy of vaccines for various age groups?

CPU times: user 19.2 s, sys: 232 ms, total: 19.5 s
Wall time: 24.2 s


'Protection against SARS-CoV-2 infection is lower among older age groups, while protection against hospitalization remains reasonably good for all age groups, among both partially and fully vaccinated'

In [155]:
print(context)



Chunk 1 trial. The latter might not be the case if the experimental treatment is suspected of being harmful and it is consequently withdrawn from current patients. Results

Chunk 2 the current study. We thank all the medical staff involved in treating the patients and all patients 161 and their families involved in this study. 171

Chunk 3 benefits still outweigh by far the potential risks, thus they should be recommended to these patients. Nevertheless, large prospective studies are needed for further investigations. Introduction

Chunk 4 remember that during the current pandemic all the medicines are used off-label, since none of them has yet been approved for the treatment of COVID-19. Introduction

Chunk 5 ⩾75% of the panel ticked the same answer. In this meeting, content of an information leaflet about treatment options for patients was discussed as well. Data collection and analysis


# Testing other QA on retrieved chunks models

##Context

In [170]:
print(query)


What were the efficacy of vaccines for various age groups?



In [168]:
print(context)



Chunk 1 of the elderly. Further studies to determine long-term effectiveness of COVID-19 vaccines in high-risk populations and the safety and effectiveness of additional boosters are needed. Main text:

Chunk 2 not always take into consideration the daily life barriers of the population. Easy access to vaccination needs to be accomplished by barrier-free access. Demographic changes In response to public health policies, life expectancy has doubled over the last century and the proportion of the elderly in the population continues to rise. Elderly people are more susceptible to infectious diseases and less responsive to vaccination, as is clearly shown by their responsiveness to the influenza vaccine (Franceschi et al. 2017; Demicheli et al. 2018; Levett-Jones 2020) . This growing situation warrants the adaptation of vaccines to the senescing immune system, thereby introducing a certain degree of personalisation at the scale of an age-defined population. For example, the Sanofi Pasteu

In [164]:
print(len(context)/4)

1100.25


##google/flan-t5-small

In [166]:
generation_model = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [167]:
%%time
# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model google/flan-t5-small will generate now to: 
What were the efficacy of vaccines for various age groups?

CPU times: user 3.02 s, sys: 148 ms, total: 3.17 s
Wall time: 4.74 s


'effective against SARS-CoV-2 infection'

##juliosocher/flan-t5-small-finetuned-scientific-articles

In [171]:
generation_model = "juliosocher/flan-t5-small-finetuned-scientific-articles"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [172]:
%%time
# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model juliosocher/flan-t5-small-finetuned-scientific-articles will generate now to: 
What were the efficacy of vaccines for various age groups?

CPU times: user 5.53 s, sys: 23.9 ms, total: 5.56 s
Wall time: 5.73 s


'Increased vaccine effectiveness against SARS-CoV-2 infection (A) and hospitalisation (B) among partially (blue) and fully (green) vaccinated individuals'

##razent/SciFive-base-Pubmed_PMC

In [173]:
generation_model = "razent/SciFive-base-Pubmed_PMC"
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [174]:
%%time
# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

Input tokenised to 994 tokens
Model razent/SciFive-base-Pubmed_PMC will generate now to: 
What were the efficacy of vaccines for various age groups?

CPU times: user 1min 19s, sys: 2.42 s, total: 1min 21s
Wall time: 1min 41s


'Background: Chunk 1 of the elderly. Both the Sanofi Pasteur Fluad trivalent flu vaccine and the Seqirus Fluad quadrivalent flu vaccine are recommended for the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 1 of the elderly. Main text: Chunk 2 of the elderly. Main text: Chunk 3 What were the efficacy of vaccines for various age groups? Answer: Yes'

??

In [ ]:
generation_model = ""
tokenizer = AutoTokenizer.from_pretrained(generation_model)
model = AutoModelForSeq2SeqLM.from_pretrained(generation_model)

In [ ]:
%%time
# Generate an answer
answer = generate_answer(query, context, model, tokenizer)
answer

# Data exploration

## Get files from the Kaggle dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("allen-institute-for-ai/CORD-19-research-challenge")

print("Path to dataset files:", path)

  4%|▎         | 680M/18.4G [00:10<04:45, 66.5MB/s]


KeyboardInterrupt: 

In [ ]:
%%time
file_dir = '/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json/'
import os
filenames = os.listdir(file_dir)
print("Number of articles retrieved from biorxiv:", len(filenames))

Number of articles retrieved from biorxiv: 401214
CPU times: user 127 ms, sys: 352 ms, total: 480 ms
Wall time: 39.1 s


In [ ]:
# select randomly 1000 article
sample_file = np.random.choice(filenames, size=1000, replace=False,)
len(sample_file)

1000

In [ ]:
%%time
all_files = []

for filename in sample_file:
    filename = file_dir + filename
    file = open(filename, 'rb')
    # TODO here would be nice to store the file locally in 'kaggle/working/'
    all_files.append(json.load(file))

CPU times: user 1.79 s, sys: 471 ms, total: 2.26 s
Wall time: 12.1 s


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pprint import pprint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



## Extract info from opened files

In [70]:
file1 = all_files[45]
print("Dictionary keys:", all_files[0].keys())

NameError: name 'all_files' is not defined

In [ ]:
texts = [(di['section'], di['text']) for di in file1['body_text']]
texts_di = {di['section']: "" for di in file1['body_text']}
for section, text in texts:
    texts_di[section] += text

#pprint(list(texts_di.keys()))

In [ ]:
pprint(file1['abstract'])

In [ ]:
def process_article(file):
    body = ""

    body += file['metadata']['title']
    body += "\n\n"
    if len(file['abstract']) > 0:
        body += file['abstract'][0]['section']
        body += "\n\n"
        body += file['abstract'][0]['text']
        body += "\n\n"

    texts = [(di['section'], di['text']) for di in file['body_text']]
    texts_di = {di['section']: "" for di in file['body_text']}
    for section, text in texts:
        texts_di[section] += text

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"

    return body



In [ ]:
docs = []
for f in all_files:
    try:
        docs.append(process_article(f))
    except:
        pprint(f)
        exit()


In [ ]:
len(docs)
print(docs[3])
article = docs[3]

# backup, early failed experiments with LLMs

In [ ]:
# Installing packages
!pip install transformers

!pip install sentence_transformers
!pip install torch
!pip install faiss-cpu

!pip install -q -U immutabledict sentencepiece

In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3




In [4]:
import kagglehub
import torch
from transformers.models.gemma3 import Gemma3ForConditionalGeneration, Gemma3Processor


In [4]:
# Importing libraries

# Importing system
#import faiss

# Importing Libraries LLM
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM


Mini data set